# Documentation for `reviews_features.ipynb`

## Overview
This notebook processes raw Amazon product review data to engineer features at the product level. The resulting features are intended for downstream analysis and modeling, such as detecting suspicious review activity or predicting product outcomes.

## Main Steps
   1) Loads raw review data from `../Data/reviews_raw.pkl`.
   2) Drops reviews missing essential fields (`summary`,`reviewText`, `reviewerName`).
   3) Computes product level statistics: average, min, count of ratings, percent positive/negative reviews, and verified purchase rates.
   4) Calculates product lifespan, reviews per day, number of unique reviewers.
   5) Flags suspicious reviewers ("bots") based on review patterns and aggregates bot counts per product.
   6) Computes review text features such as average word and character counts per product.
   7) Aggregates all engineered features at the ASIN level and saves the final dataframe to `../Data/final_reviews.parquet`.

## Input Files

- `../Data/reviews_raw.pkl` (raw Amazon product reviews)

## Output

- `../Data/final_reviews.parquet` (DataFrame with engineered features per ASIN)

In [1]:
# import relevant libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Load the Reviews Data

In [2]:
reviews_df_raw = pd.read_pickle('../Data/reviews_raw.pkl')


Look at NaNs in the Data

In [3]:
# count and percentage of missing values in each column
missing_counts = reviews_df_raw.isna().sum()
missing_percentages = reviews_df_raw.isna().mean() * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Percentage': missing_percentages
})
missing_df = missing_df[missing_df['Missing Count'] > 0]
missing_df = missing_df.sort_values(by='Missing Percentage', ascending=False)
print("Missing Values Overview:")
print(missing_df)

Missing Values Overview:
              Missing Count  Missing Percentage
image               7999253           97.537223
vote                7239598           88.274529
style               6211781           75.742056
reviewText             7117            0.086780
summary                2531            0.030861
reviewerName            502            0.006121


In [4]:
reviews_df_raw = reviews_df_raw.dropna(subset=['summary','reviewText', 'reviewerName'])

Drop Columns

In [5]:
# from this we decide to drop image, vote, style
# we will also drop reviewTime and use unixReviewTime because it is the same values but
# an easier format to work with

reviews_df_raw = reviews_df_raw.drop(columns=['image', 'vote', 'style', 'reviewTime'])

Look at Duplicates in the Data
- for now, we will keep duplicates

In [6]:
# check for duplicates in the reviews
duplicates = reviews_df_raw.duplicated(subset=['reviewerID', 'asin', 'unixReviewTime', 'overall'], keep=False)
print(f"Number of duplicate reviews: {duplicates.sum()}")
# show the duplicate reviews
duplicate_reviews = reviews_df_raw[duplicates]
duplicate_reviews[['reviewerID', 'asin', 'unixReviewTime', 'overall', 'reviewText']]

Number of duplicate reviews: 391975


,reviewerID,asin,unixReviewTime,overall,reviewText
1144,ABWSQI9992Q29,0786955554,1474761600,5,nice
1145,ABWSQI9992Q29,0786955554,1474761600,5,nice
1154,A115TZEJ3U3AER,0786955554,1462924800,5,On-time and nice item.
1155,A115TZEJ3U3AER,0786955554,1462924800,5,On-time and nice item.
1266,AN82BJBT7QNT7,0786955554,1309046400,4,This set is slightly worse than earlier ones (...
...,...,...,...,...,...
8189354,A5FVEFQ55OHRS,B01H25XJ76,1500508800,5,She is happy
8189355,A5FVEFQ55OHRS,B01H25XJ76,1500508800,5,everyone is happpy
8189356,A5FVEFQ55OHRS,B01H25XJ76,1500508800,5,The 6 year old loves it
8199065,AY6VLC56K8CVX,B01HGKDE22,1480982400,5,very pleased with promptness and quality


Convert unixReviewTime column to Datetime Objects

In [7]:
reviews_df_raw['date'] = pd.to_datetime(reviews_df_raw['unixReviewTime'], unit='s')

Features Related to Overall

In [8]:
# using simple aggregation techniques for the rating column
reviews_df_raw['avg_rating'] = reviews_df_raw.groupby('asin')['overall'].transform('mean')
reviews_df_raw['min_rating'] = reviews_df_raw.groupby('asin')['overall'].transform('min')
reviews_df_raw['min_rating'] = reviews_df_raw.groupby('asin')['overall'].transform('max')
reviews_df_raw['num_of_rating'] = reviews_df_raw.groupby('asin')['overall'].transform('count')

In [9]:
# percent positive reviews -- rating >= 4
# percent negative reviews -- rating <= 2
reviews_df_raw['is_positive'] = (reviews_df_raw['overall'] >= 4).astype(int)
reviews_df_raw['is_negative'] = (reviews_df_raw['overall'] <= 2).astype(int)

reviews_df_raw['percent_positive'] = reviews_df_raw.groupby('asin')['is_positive'].transform('mean')
reviews_df_raw['percent_negative'] = reviews_df_raw.groupby('asin')['is_negative'].transform('mean')

Features Related to Verified
- this does not seem like a great feature to use because it is so imbalanced

In [10]:
reviews_df_raw['avg_verified_reviewers'] = reviews_df_raw.groupby('asin')['verified'].transform('mean')

Features Related to UnixReviewTime

In [11]:
# compute product lifespan
reviews_df_raw['min_date'] = reviews_df_raw.groupby('asin')['date'].transform('min') 
reviews_df_raw['max_date'] = reviews_df_raw.groupby('asin')['date'].transform('max') 
reviews_df_raw['product_lifespan'] = reviews_df_raw['max_date']-reviews_df_raw['min_date']

Features Related to ReviewerID

In [12]:
# count total number of five star reviews per reviewer
all_fives = reviews_df_raw[reviews_df_raw['overall'] == 5].groupby('reviewerID').size().reset_index(name='count_of_five_star_ratings_by_reviewer')
# count total number of one star reviews per reviewer
all_ones = reviews_df_raw[reviews_df_raw['overall'] == 1].groupby('reviewerID').size().reset_index(name='count_of_one_star_ratings_by_reviewer')
#count number of total number reviews per reviewer
all_ratings = reviews_df_raw.groupby('reviewerID').size().reset_index(name='count_of_ratings_by_reviewer')

In [13]:
# merge onto one dataframe
all_ratings = all_ratings.merge(all_fives, on='reviewerID', how='left')
all_ratings = all_ratings.merge(all_ones, on='reviewerID', how='left')

In [14]:
# if reviewer gave no fives, fill with 0
all_ratings['count_of_five_star_ratings_by_reviewer'] = all_ratings['count_of_five_star_ratings_by_reviewer'].fillna(0).astype(int)
# if reviewer gave no ones, fill with 0
all_ratings['count_of_one_star_ratings_by_reviewer'] = all_ratings['count_of_one_star_ratings_by_reviewer'].fillna(0).astype(int)


In [15]:
# get proportion of fives and proportion of ones over total count
all_ratings['prop_five'] = all_ratings['count_of_five_star_ratings_by_reviewer'] / all_ratings['count_of_ratings_by_reviewer']
all_ratings['prop_one'] = all_ratings['count_of_one_star_ratings_by_reviewer'] / all_ratings['count_of_ratings_by_reviewer']
all_ratings

,reviewerID,count_of_ratings_by_reviewer,count_of_five_star_ratings_by_reviewer,count_of_one_star_ratings_by_reviewer,prop_five,prop_one
0,A0000040I1OM9N4SGBD8,1,1,0,1.000000,0.0
1,A0000862BTSWL73O3J0Y,1,1,0,1.000000,0.0
2,A00009928J2TXTYX144F,1,1,0,1.000000,0.0
3,A0001528BGUBOEVR6T5U,5,5,0,1.000000,0.0
4,A0001618R2MA7GE14QDZ,2,2,0,1.000000,0.0
...,...,...,...,...,...,...
4200112,AZZZVQOG9SG7V,1,1,0,1.000000,0.0
4200113,AZZZWV7EIR8PG,1,0,0,0.000000,0.0
4200114,AZZZYAYJQSDOJ,7,4,0,0.571429,0.0
4200115,AZZZZS162JNL0,4,4,0,1.000000,0.0


In [16]:
# find first and last date of review by reviewer
reviewer_dates = reviews_df_raw.groupby('reviewerID')['date'].agg(['min', 'max']).reset_index()
# get the difference between those dates
reviewer_dates['review_span'] = reviewer_dates['max']-reviewer_dates['min']

In [17]:
# build suspicious activity df to find potential bots
suspicious_activity_df = all_ratings.merge(reviewer_dates[['reviewerID', 'review_span']], on='reviewerID', how='left')
products_reviewed = reviews_df_raw.groupby('reviewerID')['asin'].nunique().reset_index()
products_reviewed = products_reviewed.rename(columns={'asin':'num_unique_products'})
suspicious_activity_df = suspicious_activity_df.merge(products_reviewed, on='reviewerID', how='left')
suspicious_activity_df['reviews_per_day'] = suspicious_activity_df['count_of_ratings_by_reviewer'] / (suspicious_activity_df['review_span'].dt.days + 1 )


In [18]:
# make bots df by specifying criteria based on distribution of each feature
bots_df = suspicious_activity_df[
    (suspicious_activity_df['count_of_ratings_by_reviewer'] > 10) & 
    (suspicious_activity_df['reviews_per_day'] > 3) & 
    (suspicious_activity_df['num_unique_products'] > 10) & 
    ((suspicious_activity_df['prop_five'] == 1) | (suspicious_activity_df['prop_one'] > 0.8))
]

In [19]:
# put bot count on reviews_data_df
bots = list(bots_df['reviewerID'].unique())
reviews_df_raw['is_bot'] = 0
reviews_df_raw.loc[reviews_df_raw['reviewerID'].isin(bots), 'is_bot'] = 1

reviews_df_raw['num_bots_per_asin'] = reviews_df_raw.groupby('asin')['is_bot'].transform('sum')

In [20]:
suspicious_activity_df.columns

Index(['reviewerID', 'count_of_ratings_by_reviewer',
       'count_of_five_star_ratings_by_reviewer',
       'count_of_one_star_ratings_by_reviewer', 'prop_five', 'prop_one',
       'review_span', 'num_unique_products', 'reviews_per_day'],
      dtype='object')

In [21]:
reviews_df_raw = reviews_df_raw.merge(suspicious_activity_df[['reviewerID', 'reviews_per_day']], on='reviewerID', how='left')

In [22]:
# number of unique reviewers per product
reviews_df_raw['unique_reviewer_count'] = reviews_df_raw.groupby('asin')['reviewerID'].transform('nunique')

In [23]:
reviews_df_raw['avg_reviews_per_day'] = reviews_df_raw.groupby('asin')['reviews_per_day'].transform('mean')

Features Related to ReviewText

In [24]:
# number of reviews per product
reviews_df_raw['reviews_per_product'] = reviews_df_raw.groupby('asin')['reviewText'].transform('count')

In [ ]:
# word count of each review
reviews_df_raw['review_length_words'] = reviews_df_raw['reviewText'].str.split().str.len()

# character count of each review
reviews_df_raw['review_length_chars'] = reviews_df_raw['reviewText'].str.len()

# aggregate the above columns
reviews_df_raw['avg_review_length_words'] = reviews_df_raw.groupby('asin')['review_length_words'].transform('mean')
reviews_df_raw['avg_review_length_chars'] = reviews_df_raw.groupby('asin')['review_length_chars'].transform('mean')


Make Final Reviews DF grouped by ASIN

In [ ]:
final_reviews_df = reviews_df_raw[['asin', 'avg_rating', 'min_rating', 'num_of_rating','percent_positive', 'percent_negative', 'avg_verified_reviewers',
                'min_date', 'max_date', 'product_lifespan', 'num_bots_per_asin', 'unique_reviewer_count', 'avg_reviews_per_day', 'reviews_per_product',
                'avg_review_length_words', 'avg_review_length_chars' ]].drop_duplicates()

In [ ]:
final_reviews_df.isna().sum()

asin                       0
avg_rating                 0
min_rating                 0
num_of_rating              0
percent_positive           0
percent_negative           0
avg_verified_reviewers     0
min_date                   0
max_date                   0
product_lifespan           0
num_bots_per_asin          0
unique_reviewer_count      0
avg_reviews_per_day        0
reviews_per_product        0
avg_review_length_words    0
avg_review_length_chars    0
dtype: int64

In [ ]:
final_reviews_df

,asin,avg_rating,min_rating,num_of_rating,percent_positive,percent_negative,avg_verified_reviewers,min_date,max_date,product_lifespan,num_bots_per_asin,unique_reviewer_count,avg_reviews_per_day,reviews_per_product,avg_review_length_words,avg_review_length_chars
0,0020232233,4.130435,5,23,0.739130,0.173913,0.782609,2016-09-12,2018-04-06,571 days,0,23,0.642518,23,59.260870,329.608696
13,038536539X,3.615385,5,13,0.538462,0.307692,0.923077,2017-01-26,2018-03-30,428 days,0,13,1.309957,13,29.769231,157.923077
16,0486277577,4.357143,5,14,0.857143,0.071429,0.500000,2000-10-15,2015-05-31,5341 days,0,14,0.513082,14,105.571429,599.928571
20,0486402029,3.687500,5,16,0.625000,0.250000,0.625000,2005-09-20,2016-02-04,3789 days,0,16,0.568113,16,52.687500,275.312500
26,0486427706,4.800000,5,40,0.925000,0.000000,0.950000,2007-02-06,2017-07-16,3813 days,0,40,0.534902,40,22.850000,131.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8191237,B01HJBANAE,4.833333,5,6,1.000000,0.000000,0.833333,2017-04-28,2018-01-30,277 days,0,6,0.342315,6,36.500000,200.500000
8191263,B01HJBAKGQ,5.000000,5,2,1.000000,0.000000,1.000000,2017-11-13,2018-04-22,160 days,0,2,0.024517,2,25.500000,122.000000
8191265,B01HJAEYBY,4.666667,5,3,1.000000,0.000000,0.333333,2016-09-18,2016-10-06,18 days,0,3,0.346884,3,110.666667,565.666667
8191268,B01HJDGVFS,4.200000,5,5,0.800000,0.200000,1.000000,2016-08-26,2018-08-13,717 days,0,5,0.603825,5,11.600000,57.400000


In [ ]:
final_reviews_df.to_parquet("../Data/final_reviews.parquet")